In [1]:
import glob

wav_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_data/*.wav"))
csv_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_labels/*.csv"))

In [ ]:
import torch
import librosa
import numpy as np
from Model.CNN_3layer import *

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = CNN_3L().to(device)
checkpoint = torch.load("../Trained/checkpoint.pth", weights_only=True, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
for name, param in model.named_parameters():
	if "fc2" in name:
		print(name, param.mean().item(), param.std().item())
model.eval()

# Dùng model để dự đoán nốt nhạc của một file WAV
wav_file = "../Data/MusicNet_Dataset/musicnet/musicnet/train_data/2478.wav"
predicted_notes = predict_notes(model, device, wav_file)
print("Predicted notes:", predicted_notes)
print("Unique predicted values:", np.unique(predicted_notes))

for note, start, end in predicted_notes:
	if note == 128:  # Nếu là nhãn "không có nốt", bỏ qua
		continue
	note_name = librosa.midi_to_note(note)  # Chuyển số MIDI thành tên nốt nhạc
	print(f"Note: {note_name}, Start: {start:.2f}s, End: {end:.2f}s")

In [3]:
import librosa
import numpy as np

y, sr = librosa.load("../Data/MusicNet_Dataset/musicnet/musicnet/test_data/1819.wav")

# Phát hiện beat và downbeat
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
onset_env = librosa.onset.onset_strength(y=y, sr=sr)

# Chuyển beats thành thời gian
beat_times = librosa.frames_to_time(beats, sr=sr)

if len(beat_times) > 1:
	# Tính khoảng cách giữa các beats
	beat_intervals = np.diff(beat_times)
	# Tìm khoảng cách phổ biến nhất giữa các beats mạnh
	avg_interval = np.median(beat_intervals)

	# Xác định nhịp theo độ lặp lại của downbeat
	if avg_interval < 0.4:  # Nếu khoảng cách nhỏ (thường là 3 beats trong 1 chu kỳ)
		time_signature = "3/4"
	else:  # Nếu khoảng cách lớn hơn (thường là 4 beats trong 1 chu kỳ)
		time_signature = "4/4"
else:
	time_signature = "Không xác định"

# In kết quả
print(f"Tempo: {tempo[0]:.2f} BPM")
print(f"Ước lượng nhịp: {time_signature}")

Tempo: 152.00 BPM
Ước lượng nhịp: 3/4


In [ ]:
import librosa
import glob
import numpy as np

path = "../Data/MusicNet_Dataset/musicnet/musicnet/train_data/*.wav"

files = glob.glob(path)

unique_tempos = set()

for file in files:
	y, sr = librosa.load(file)
	tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
	tempo = tempo[0] if tempo.ndim > 0 else tempo
	# Thêm tempo vào tập hợp
	unique_tempos.add(round(tempo, 2))

print("Các tempo duy nhất:")
for t in sorted(unique_tempos):
	print(f"{t}, ")

In [ ]:
import mido
import glob

path = "../Data/MusicNet_Dataset/musicnet_midis/musicnet_midis/Bach/2302_fugue5.mid"

files = glob.glob(path)

unique_time_signatures = set()

for file in files:
	try:
		midi = mido.MidiFile(file, clip=True)  # clip=True giúp bỏ qua giá trị không hợp lệ
		for track in midi.tracks:
			for msg in track:
				if msg.type == 'time_signature':
					time_sig = f"{msg.numerator}/{msg.denominator}"
					unique_time_signatures.add(time_sig)
	except Exception as e:
		print(f"Lỗi khi đọc file {file}: {e}")

print("Các nhịp có trong dữ liệu MIDI:")
for ts in sorted(unique_time_signatures):
	print(ts)